In [46]:
#-*-coding:utf-8 -*-
#所有需要的库文件添加该处
import pandas as pd
import  numpy as np
import os
import sys
from datetime import datetime,timedelta,date
from pyecharts import Line,Kline,Bar,Overlap,Grid
from jqdatasdk import *

sys.path.append('C:\\quanttime\\src\\comm\\')
import trade_date_util

pd.set_option('precision',3)
auth('13811866763', "sam155")  # jqdata 授权

In [5]:
index_dir = "C:\\quanttime\\data\\index\\jq\\"
# 获取所有的指数标的
index_security = get_all_securities(types=['index'], date=None)
# 滤除退市的，或者当前不存在的指数
index_security = index_security[index_security["end_date"] == "2200-01-01"]
index_security.head(2)

,display_name,name,start_date,end_date,type
000001.XSHG,上证指数,SZZS,1991-07-15,2200-01-01,index
000002.XSHG,A股指数,AGZS,1992-02-21,2200-01-01,index


In [41]:
# 设置比较基准,front_buy:与前front_buy进行比较，如果高则买入，与end_sell比较，如果低则卖出
front_buy = 20
end_sell = 20

In [6]:
# 获取所有交易日 [numpy.ndarray],元素为datetime.date
trade_date = get_all_trade_days()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [42]:
def get_trade_signal(code, date):
    '''
    获取交易信号，输入指数的code，起始日期
    例如获取date=2019-6-1 的交易信号
    '''
    file = index_dir + code + '.csv'
    df = pd.read_csv(file, index_col=["date"], parse_dates=True, usecols=["date", "close"])
    
    list_date = get_trade_date_by_count(date, front_buy)
    pre_20 = list_date[0]
    pre_1 = list_date[-2]
    if (pre_1 not in df.index) or (pre_20 not in df.index):
        return "unkown"
    try:
        close_1 = df.loc[pre_1,["close"]]["close"]
        close_20 = df.loc[pre_20,["close"]]["close"]
    except KeyError:
        return "unkown"
    if close_1 > close_20:
        return "buy"
    elif close_1 < close_20:
        return "sell"
    else:
        return "hold"
    

In [35]:
def process_trade_result(list_record):
    '''
    处理交易结果
    '''
    list_trade_point = []
    for pos, dic in enumerate(list_record):   
        dic_key = dic.keys()[0]
        dic_value = dic.values()[0]
        trade_direct = dic_key.split('_')[0]
        trade_date = dic_key.split('_')[1]
        if pos == 0:
            if trade_direct != "buy":
                print("本次交易记录有误，record：%r" % list_record)
                return
        if pos % 2 == 0:
            if trade_direct != "buy":
                print("本次交易记录有误，record：%r" % list_record)
                return
        else:
            if trade_direct != "sell":
                print("本次交易记录有误，record：%r" % list_record)
                return
        list_trade_point.append(dic_value)
    list_diff = []
    for i in range(int(len(list_trade_point) / 2)):
        front = list_trade_point.pop(0)
        end = list_trade_point(0)
        diff = (end - front)/ front
        list_diff.append(diff)
    result = 1
    for i in list_diff:
        result = result * (1 + i)
    return result      

In [47]:
def get_trade_date_by_count(start_date, count):
    '''
    根据指定的起始日期，往前推count天
    这里往前推的日期不包括start_date，也就是从start_date的前一天开始计算
    :param start_date: datetime，date，str
    :param count: int 天
    :return: list date类型
    '''
    start = to_date(start_date)
    if start is None:
        return start_date
    trade_date_file = r'C:\quanttime\data\basic_info\all_trade_day.csv'
    trade_date = pd.read_csv(trade_date_file, index_col=["trade_date"], parse_dates=True)
    list_trade_date = trade_date.index.tolist()
    list_trade_date = [d.date() for d in list_trade_date]
    return [d for d in list_trade_date if d <= start][-count-1:]

# =========================


def to_date(input_date):
    '''
    convert_date('2015-1-1')
    datetime.date(2015, 1, 1)

    convert_date('2015-01-01 00:00:00')
    datetime.date(2015, 1, 1)

    convert_date(datetime.datetime(2015, 1, 1))
    datetime.date(2015, 1, 1)

    convert_date(datetime.date(2015, 1, 1))
    datetime.date(2015, 1, 1)
    '''
    if is_str(input_date):
        if ':' in input_date:
            input_date = input_date[:10]
        return datetime.strptime(input_date, '%Y-%m-%d').date()
    elif isinstance(input_date, datetime):
        return input_date.date()
    elif isinstance(input_date, date):
        return input_date
    elif input_date is None:
        return None


def is_str(s):
    return isinstance(s, str)

In [48]:
list_trade_result_record = []
# position:0 空仓，1 持仓
position = 0
for code in index_security.index[0:2]:
    start_date = index_security.loc[code,["start_date"]]["start_date"]
    start = start_date + timedelta(days=180)
    file = index_dir + code + '.csv'
    df = pd.read_csv(file, index_col=["date"], parse_dates=True, usecols=["date", "open"])
    if start > datetime.today():
        print("指数开始的时间过于接近当前日期，暂时不统计，start_date:%s" % str(start_date))
        continue
    yesterday = datetime.today() - timedelta(days=1)
    print(start)
    print(yesterday)
    list_trade_date = trade_date_util.get_trade_date_range2(start, yesterday)
    for st in list_trade_date:
        signal = get_trade_signal(code, st)
        if (signal == "buy") and (position == 0):
            try:
                buy_price = df.loc[st,["open"]]["open"]
                list_trade_result_record.append({'buy_' + str(st): buy_price})
            except KeyError:
                print("date:%s 没有对应的指数值" % str(st))
        elif (signal == "sell") and (position == 1):
            try:
                sell_price = df.loc[st,["open"]]["open"]
                list_trade_result_record.append({'sell_' + str(st): sell_price})
            except KeyError:
                print("date:%s 没有对应的指数值" % str(st))
        else:
            print("date:%s 指数相对20天前没有变化或获取的状态未知" % str(st))
    result = process_trade_result(list_trade_result_record)
    print(result)
            
    
    
    

1992-01-11 00:00:00
2019-06-16 16:25:05.239882


IndexError: list index out of range

In [51]:
tmp = [1,2,3]
tmp[-4]

IndexError: list index out of range